In [1]:
import sys
import platform
print("Python version:\n", sys.version)
print ("Path to the python executable:\n", sys.executable)

Python version:
 3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 14:00:49) [MSC v.1915 64 bit (AMD64)]
Path to the python executable:
 C:\Users\sneha\.conda\envs\decathlon\python.exe


# Biomedical Image Segmentation with U-Net

In this code example, we apply the U-Net architecture to segment brain tumors from raw MRI scans as shown below. With relatively little data we are able to train a U-Net model to accurately predict where tumors exist. 

The Dice coefficient (the standard metric for the BraTS dataset used in the study) for our model is about 0.82-0.88.  Menze et al. [reported](http://ieeexplore.ieee.org/document/6975210/) that expert neuroradiologists manually segmented these tumors with a cross-rater Dice score of 0.75-0.85, meaning that the model’s predictions are on par with what expert physicians have made.

<img src="images/figure1.png">

Since its introduction two years ago, the [U-Net](https://arxiv.org/pdf/1505.04597.pdf0) architecture has been used to create deep learning models for segmenting [nerves](https://github.com/jocicmarko/ultrasound-nerve-segmentation) in ultrasound images, [lungs](https://www.kaggle.com/c/data-science-bowl-2017#tutorial) in CT scans, and even [interference](https://github.com/jakeret/tf_unet) in radio telescopes.

## What is U-Net?
U-Net is designed like an [auto-encoder](https://en.wikipedia.org/wiki/Autoencoder). It has an encoding path (“contracting”) paired with a decoding path (“expanding”) which gives it the “U” shape.  However, in contrast to the autoencoder, U-Net predicts a pixelwise segmentation map of the input image rather than classifying the input image as a whole. For each pixel in the original image, it asks the question: “To which class does this pixel belong?” This flexibility allows U-Net to predict different parts of the tumor simultaneously.

<img src="images/unet.png">

This module loads the data from `data.py`, creates a TensorFlow/Keras model from `model.py`, trains the model on the data, and then saves the best model.

In [2]:
import tensorflow as tf 
import datetime
import time
import os
from IPython.display import Image

import numpy as np

import settings    # Use the custom settings.py file for default parameters
# import onnxmltools

import sys; sys.argv=['']; del sys
# from model import load_model, get_callbacks, evaluate_model
from data import load_data
from model import unet

from argparser import args

if args.keras_api:
    import keras as K
else:
    from tensorflow import keras as K

print ("We are using Tensorflow version", tf.__version__,\
       "with Intel(R) MKL", "enabled" if tf.pywrap_tensorflow.IsMklEnabled() else "disabled",)
print("Keras API version: {}".format(K.__version__))

onnx = False # Set whether we are exporting to ONNX model and using nGraph

We are using Tensorflow version 1.11.0 with Intel(R) MKL enabled
Keras API version: 2.2.4


Using TensorFlow backend.


For best CPU speed set the number of intra and inter threads to take advantage of multi-core systems.
See https://github.com/intel/mkl-dnn

In [3]:
#TODO: Set the multi-threading parameters for Tensorflow. 
config = tf.ConfigProto(intra_op_parallelism_threads=args.num_threads,
                        inter_op_parallelism_threads=args.num_inter_threads)

sess = tf.Session(config=config)

K.backend.set_session(sess)

## Training Time!

The bulk of the training section can be broken down in 4 simple steps:
1. Load the training data
1. Define the model
3. Train the model on the data
4. Evaluate the best model
    

#### Step 1 : Loading the datafrom the `HDF5` file

In [4]:
hdf5_filename = os.path.join(args.data_path, args.data_filename)
print ("Full path to the HFS file:", hdf5_filename)

Full path to the HFS file: ../../data/decathlon/144x144/Task01_BrainTumour.h5


In [5]:
imgs_train, msks_train, imgs_validation, msks_validation, imgs_testing, msks_testing = \
    load_data(hdf5_filename, args.batch_size,[args.crop_dim, args.crop_dim])

np.random.seed(816)

Batch size = 128
Training image dimensions:   (58464, 144, 144, 4)
Training mask dimensions:    (58464, 144, 144, 1)
Validation image dimensions: (4608, 144, 144, 4)
Validation mask dimensions:  (4608, 144, 144, 1)
Testing image dimensions: (6624, 144, 144, 4)
Testing mask dimensions:  (6624, 144, 144, 1)


#### Step 2: Define the model

In [6]:
print("-" * 30)
print("Creating and compiling model ...")
print("-" * 30)
unet_model = unet()
model = unet_model.create_model(imgs_train.shape, msks_train.shape)

model_filename, model_callbacks = unet_model.get_callbacks()

# # If there is a current saved file, then load weights and start from there.
# saved_model = os.path.join(args.output_path, args.inference_filename)
# if os.path.isfile(saved_model):
#     model.load_weights(saved_model)

------------------------------
Creating and compiling model ...
------------------------------
Data format = channels_last
Using UpSampling2D
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
MRImages (InputLayer)           (None, 144, 144, 4)  0                                            
__________________________________________________________________________________________________
encodeAa (Conv2D)               (None, 144, 144, 32) 1184        MRImages[0][0]                   
__________________________________________________________________________________________________
encodeAb (Conv2D)               (None, 144, 144, 32) 9248        encodeAa[0][0]                   
__________________________________________________________________________________________________
poolA (MaxPooling2D)            (None, 72, 72, 32)   0           e

The code snippet below draws the model using Keras' built-in `plot_model`. Compare with the implementation of `model.py`

In [7]:
tf.keras.utils.plot_model(model,
                          to_file='images/model.png',
                          show_shapes=True,
                          show_layer_names=True,
                          rankdir='TB'
                         )
Image('images/model.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

#### Step 3: Train the model on the data

In [8]:
print("-" * 30)
print("Fitting model with training data ...")
print("-" * 30)

print("Step 3, training the model started at {}".format(datetime.datetime.now()))
start_time = time.time()

if onnx:
    for i in range(0,30):
        print ("Processing batch:", i)
        start_time = time.time()
        model.train_on_batch(imgs_train[i*args.batch_size:args.batch_size*(i+1)-1], \
                             msks_train[i*args.batch_size:args.batch_size*(i+1)-1])
        print ("Time for training on batch:", time.time() - start_time) 

        # TODO: Convert the Keras model to ONNX and save it. 
        onnx_model = onnxmltools.convert_keras(model, target_opset=7) 
        onnxmltools.utils.save_model(onnx_model, 'output/unet_model_for_decathlon.onnx')
else:
    history = model.fit(imgs_train, msks_train,
              batch_size=args.batch_size,
              epochs=args.epochs,
              validation_data=(imgs_validation, msks_validation),
              verbose=1, shuffle="batch",
              callbacks=model_callbacks)
    
print("Total time elapsed for training = {} seconds".format(time.time() - start_time))
print("Training finished at {}".format(datetime.datetime.now()))
    
# Append training log
# with open("training.log","a+") as fp:
#     fp.write("{}: {}\n".format(datetime.datetime.now(),
#                              history.history["val_dice_coef"]))

------------------------------
Fitting model with training data ...
------------------------------
Step 3, training the model started at 2019-08-06 01:00:23.673330
Train on 58464 samples, validate on 4608 samples
Epoch 1/30
  128/58464 [..............................] - ETA: 18:50:36 - loss: 2.9644 - acc: 0.1780 - dice_coef: 0.0397 - soft_dice_coef: 0.0372

KeyboardInterrupt: 

#### Step 4: Evaluate the best model

In [ ]:
print("-" * 30)
print("Loading the best trained model ...")
print("-" * 30)
unet_model.evaluate_model(model_filename, imgs_testing, msks_testing)

## End: In this tutorial, you have learnt:
* What is the U-Net model
* Comparing training times - Tensorflow_MKLDNN vs Tensorflow (stock)
* How to tweak a series of environment variables to get better performance out of MKLDNN
* How to tweak a series of Tensorflow-related and neural-network specific parameters for better performance

`Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License. SPDX-License-Identifier: EPL-2.0`

`Copyright (c) 2019 Intel Corporation`